In [ ]:
################################################################################
# Author 1:      Firstname Lastname
# MatNr 1:       01234567
# Author 2:      Firstname Lastname
# MatNr 2:       01234567
# Author 3:      Jakob Marktl
# MatNr 3:       12335939
# File:          assignment2.ipynb
# Description: ... short description of the file ...
# Comments:    ... comments for the tutors ...
#              ... can be multiline ...
################################################################################

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import pycountry
import pycountry_convert

Alles in einem File, programmieren

Robert:

David:

In [1]:
print("David hat was gemacht")

David hat was gemacht


Jakob:

In [38]:
FishBoats = pd.read_csv('Fish_Boote.csv')
FishInland = pd.read_csv('Fish_Inland.csv')

# FISH BOATS
# transfering mentionable values into lists
boat_referenceArea = FishBoatsSorted['Reference area'].tolist()
boat_timePeriod = FishBoatsSorted['TIME_PERIOD'].tolist()
boat_obsValue = FishBoatsSorted['OBS_VALUE'].tolist()

# Creating DataFrame
boat_Data = pd.DataFrame({
    'country': boat_referenceArea,
    'absolute_value': boat_obsValue,
    'timePeriod': boat_timePeriod
})
# FISH INLAND

# Transfering mentionable values into lists
inland_referenceArea = FishInlandsorted['Reference area'].tolist()
inland_timePeriod = FishInlandsorted['TIME_PERIOD'].tolist()
inland_obsValue = FishInlandsorted['OBS_VALUE'].tolist()

# Creating DataFrame
inland_Data = pd.DataFrame({
    'country': inland_referenceArea,
    'absolute_value': inland_obsValue,
    'timePeriod': inland_timePeriod
})

# Function for Bar Chart
def create_casual_chart(data, title, year_filter):

    # set Filter to 2010
    filtered_Data = data[data['timePeriod'] == year_filter]

    # Scatter Plot using plotly.express
    fig = px.bar(filtered_Data, x='country', y='absolute_value', 
                    title=title,
                    labels={'Time_Period': 'Time Period (Years)', 'absolute_value': 'Amount of vessels'})
    # logahritmic scale:
    fig.update_yaxes(type='log')
    fig.update_layout(
        xaxis=dict(tickangle=45)  # Rotates x-axis labels by 45 degrees
    )
    # Show the plot
    fig.show()

# Create chart for Boats
#create_casual_chart(boat_Data,"Scatter Plot of amount of fishing vessels in each Country for 2010",2010)
# Create chart for catches
#create_casual_chart(inland_Data,"Scatter Plot of amount of fishing vessels in each Country for 2010",2010)




# Mixed Scatter Plot

# finding common countries & time:

common_countries = set(boat_Data['country']).intersection(inland_Data['country'])
common_time = set(boat_Data[boat_Data["country"].isin(common_countries)]['timePeriod']
                    ).intersection(inland_Data[inland_Data["country"].isin(common_countries)]['timePeriod'])

# Creating a full index of common countries and time periods
full_index = pd.MultiIndex.from_product(
    [common_countries, common_time], names=["country", "timePeriod"]
)

# Reindexing both datasets and filling missing values with 0
boat_Data_aligned = (
    boat_Data_final.set_index(["country", "timePeriod"])
    .reindex(full_index)
    .fillna(0)
    .reset_index()
)
inland_Data_aligned = (
    inland_Data_final.set_index(["country", "timePeriod"])
    .reindex(full_index)
    .fillna(0)
    .reset_index()
)

# Merging the two DataFrames
combined_data = pd.merge(
    boat_Data_aligned, 
    inland_Data_aligned, 
    on=["country", "timePeriod"], 
    suffixes=("_boat", "_inland")  # Add suffixes to distinguish columns
)
print(combined_data)

combined_data_timefiltered = combined_data[combined_data['timePeriod'] == 2010]

 # Scatter Plot using plotly.express
fig = px.scatter(
    combined_data_timefiltered, 
    x='absolute_value_boat', 
    y='absolute_value_inland', 
    title="Amount of Vessels vs Amount of Inland Fish Catches in 2010",
    labels={'timePeriod': 'Time Period (Years)', 'absolute_value_boat': 'Amount of vessels', 'absolute_value_inland': 'Amount of catches'},
    color='country'
    )
# logahritmic scale:
fig.update_yaxes(type='log')
fig.update_xaxes(type='log')
fig.update_layout(
    xaxis=dict(tickangle=45)  # Rotates x-axis labels by 45 degrees
 )
# Show the plot
fig.show()


     country  timePeriod  absolute_value_boat  absolute_value_inland
0    Türkiye        2016              15663.0                33370.0
1    Türkiye        2017              15406.0                31598.0
2    Türkiye        2018              15352.0                29652.0
3    Türkiye        2019              15315.0                31142.0
4    Türkiye        2020              15302.0                32829.0
..       ...         ...                  ...                    ...
475     Peru        2011                  0.0                36766.0
476     Peru        2012                  0.0                33601.0
477     Peru        2013                  0.0                27301.0
478     Peru        2014                  0.0                24682.0
479     Peru        2015                  0.0                37499.0

[480 rows x 4 columns]
